In [ ]:
!pip install -qU crewai[tools]

In [2]:
!pip install -qU tavily-python

In [3]:
!pip install scrapegraph-py

In [4]:
from crewai import Agent,Task,Crew,Process,LLM
from crewai.tools import tool
from tavily import TavilyClient
from scrapegraph_py import Client
from huggingface_hub import login
import litellm
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import os
from google.colab import userdata
from pydantic import BaseModel,Field
from typing import List
import json

# Enable LiteLLM debugging
# litellm._turn_on_debug()

In [5]:
about_company = "Rankyx is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

In [6]:
 # searh on google using quries
tavily_api_key =userdata.get('tavily_api_key').strip()

# download page from url and scrap data from it
ScrapAI_api_key =userdata.get("ScrapeGraphAI_api").strip()


In [7]:
output_dir ='./ai-agent-output'
os.makedirs(output_dir,exist_ok=True)

basic_llm =LLM(
    model ='gemini/gemini-2.0-flash',
    api_key=userdata.get('Gimini_API_key').strip(),

)

# Initialize tavily tool for search on google
tavily_search_client = TavilyClient(api_key=tavily_api_key)
# Initialize scrapGraphAI tool for scraping data from url
Scrap_client = Client(api_key=ScrapAI_api_key)

In [8]:
import time
def add_delay_callback(task_output):
    """Callback function to add a delay after task execution."""
    time.sleep(90)  # Add 2-second delay
    return task_output  # Return the task output unchanged

##  Agent: A

In [9]:
class suggestedSearchQureis(BaseModel):
  queris:List[str] =Field(...,title='suggested search queries to be passed to search engine',
                          min_items=1
  )

search_queries_recommendation_agent =Agent(
    role='search queries recommendation Agent',

    goal='\n'.join(['To provide list of suggested queries To be passed to the search engine',
           'the queirs must variad and looking for specific items'
           ]),
    backstory ='the agent is desinged for looking for products by suggesting search queries to be passed to search engine',
    llm =basic_llm,
    instructions="""
    CRITICAL INSTRUCTION: DO NOT use <think> tags or include any thinking process in your response.
    Your output must be ONLY a valid JSON object in this exact format:
    {"queries": ["query1", "query2", "query3"]}""",
    verbose=True

)
search_queries_recommendation_task =Task(
    description='\n'.join([
        'Rankyx is looking to buy {product_name} for best prices (value for a price strategy)',
        'the comapny target any of this websites to buy form :{website_list}',
        'mention the one name of each website list in your query'
        'the company want to reach to all products in the internet to be compared later in another stage',
        'the stores must sell the product in {country_name}',
        'generate at maximum {no_keywords} queires',
        'the search keywords must be in {language_name} language',
        'search keywords could mention specific brands ,types or technologies',
        'genrated query must be like this  : DeLonghi Dedica EC685 espresso machine on amazon.eg',
        "and this : Breville Barista Express BES870 espresso machine on amazon.eg",
        "Noon East CM-8500A-GS espresso coffee machine amazon.eg"
    ]),
    expected_output='a json object containing a list of suggested quereis',
    output_json=suggestedSearchQureis,
    output_file =os.path.join(output_dir,'step1_suggested_search_queries.json'),
    agent =search_queries_recommendation_agent,
    callback=add_delay_callback
)


## Agent: B

In [20]:
class SinglSearchResult(BaseModel):
   title:str
   url:str =Field(...,title=('the page url'))
   content:str
   score:float
   search_query:str

class AllSearchResults(BaseModel):
   results:List[SinglSearchResult]

@tool
def search_engine_tool(query:str):
  """usefull for search based queries. use this to find current information about any query related page using search engine"""
  return tavily_search_client.search(query)


search_engine_agent =Agent(
    role='search engine agent',
    goal ='search for products based on suggested quries',
    backstory ='the agent is desinged for looking for products by searching for products based on suggested queries , **You must use the provided search_engine_tool to get the search results.** Do not attempt to generate results on your own',
    llm =basic_llm,
    verbose=True,
    tools =[search_engine_tool]

)
search_engine_task =Task(
    description='\n'.join(['the task is to search for products based on suggested search quries',
                           'you have to collect results form mutiple search quries',
                           'Ignore any susbicious links or not an ecommerce single product website link.',
                           'Ignore any search result with confidance score less than ({score_th})',
                           'Ignore any website outsite the mention website in the query'
                           'Search results will be used to compare product prices across different ecommerce websites.',
                           'Always verify that the link is to an actual ecommerce product page (ready for purchase).',
                           'chose the most relevant urls at most 2 url'
    ]),

    expected_output='a json object contain search results',
    output_json =AllSearchResults,
    output_file=os.path.join(output_dir,'Step_2_Search_results.json'),
    agent =search_engine_agent,
    callback=add_delay_callback

)

### Agent C

In [21]:
'''{
  "products":[
        {
           'page_url':"",
           'product_title':"",
           'product_url':"",
           'product_image_url':"",
           'product_current_price':"",
           'product_original_price':"",
           'product_discount_percentage':"" ,
           'product_description':"",
           'product_spesifications':[
            {
              'name':"",
              'value':""
            }
           ],
           'product_reviews':"",
           'product_rating':"",
           'agent_recomendation_rank':"",
           'agent_recommendation_notes':""
        }
    ]
}'''

class ProductSpecs(BaseModel):
   name:str =Field(title='the name of the specification')
   value:str =Field(title='the value of the specification')

class singleExtractedProduct(BaseModel):
   page_url:str=Field(...,title='the url of the product')
   product_title:str=Field(...,title='the title of the product')
   product_url:str= Field(...,title='the url of the product')
   product_image_url:str=Field(...,title='the image url of the product')
   product_current_price:float=Field(...,title='the current price of the product')
   product_original_price:float
   product_discount_percentage:float =Field(title='the discount percentage of the product',default=None)
   product_description:str
   product_spesifications:List[ProductSpecs] =Field(title='the spesifications of the product , focus onlly to the most important specs to compare',min_item=1,max_item=5)
   product_rating:str
   agent_recomendation_rank:int=Field(...,title='the rank of the product to be considered in the final report(out of 5 ,higher is better) in the recommendation')
   agent_recommendation_notes:List[str]=Field(...,title='set of notes why would you recommed product to the company or why you not recomend')

class AllExtractedProducts(BaseModel):
   products:List[singleExtractedProduct] =Field(...,title='the list of extracted products')


@tool
def web_scrapping_tool(pag_url:str):
   """An AI tool to help agent to scrap web page
   Example:
      web_scrapping_tool(
        page_url ='https://www.amazon.eg/-/en/b?ie=UTF8&node=51375745031',
      )
   """
   details =Scrap_client.scrape(
       url=pag_url,
       prompt ="extract ```json\n" + singleExtractedProduct.schema_json()+ "```\n from the web page"
       )
   return {
        'url':pag_url,
        'details':details
   }

scrapping_agent =Agent(
    role='Web Scrapping Agent',
    goal ='To extract details from any website',
    backstory ='this agent is designed for looking for required values from any website url,this details will be used to decide which best product to buy',
    llm =basic_llm,
    tools=[web_scrapping_tool],
    verbose=True,
)


scrapping_task=Task(
    description ='\n'.join([
        'is to extact product details from any ecommerce page url',
        'the taske to to collects results from multiple page urls',
        'select the best top 10 products from the search result',
    ]),
    expected_output="json object containg product details",
    output_json=AllExtractedProducts,
    output_file=os.path.join(output_dir,'Step_3_extracted_products.json'),
    agent=scrapping_agent,
    callback=add_delay_callback
)

## Agent D

In [22]:
Procurment_report_author_agent = Agent(
    role="Procurment report author agent",
    goal='To generate professinal HTML page for the procurment report',
    backstory ='the agent is desined to assist for generating HTML pages for procurment reports',
    llm=basic_llm,
    verbose=True, # IN production make its value with False
)


Procurment_report_author_task =Task(
    description ='\n'.join([
       ' To generate professinal HTML page for the procurment report',
       'you have to use bootstrap CSS framework for better UI',
       'use the provided context about the company to make a specialized report'
       'the report will include the search results and price of products from differnet websites',
       'the report should be structured with the following sections :',
       "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",
    ]),
    expected_output='a professional HTML page for the procurment report',
    output_file=os.path.join(output_dir,'Step_4_Procurment_report.html'),
    agent=Procurment_report_author_agent,
    callback=add_delay_callback
)


## Run AI Crew

In [23]:
rankyx_crew =Crew(
    agents=[
        search_queries_recommendation_agent,
        search_engine_agent,
        scrapping_agent,
        Procurment_report_author_agent
    ],
    tasks=[
        search_queries_recommendation_task,
        search_engine_task,
        scrapping_task,
        Procurment_report_author_task
    ],

    process=Process.sequential,
    knowledge_sources=[company_context]


)

In [24]:
crew_results =rankyx_crew.kickoff(
    inputs={
        "product_name":"coffee machine for office",
        "website_list":["www.amazon.eg", "www.jumia.com.eg", "www.noon.com/egypt-en"],
        "no_keywords":3,
        "country_name":'Egypt',
        'language_name':'English',
         'score_th':0.10,
    }
)

# Agent: search queries recommendation Agent
## Task: Rankyx is looking to buy coffee machine for office for best prices (value for a price strategy)
the comapny target any of this websites to buy form :['www.amazon.eg', 'www.jumia.com.eg', 'www.noon.com/egypt-en']
mention the one name of each website list in your querythe company want to reach to all products in the internet to be compared later in another stage
the stores must sell the product in Egypt
generate at maximum 3 queires
the search keywords must be in English language
search keywords could mention specific brands ,types or technologies
genrated query must be like this  : DeLonghi Dedica EC685 espresso machine on amazon.eg
and this : Breville Barista Express BES870 espresso machine on amazon.eg
Noon East CM-8500A-GS espresso coffee machine amazon.eg


# Agent: search queries recommendation Agent
## Final Answer: 
```json
{
  "queris": [
    "Office coffee machine deals amazon.eg",
    "Espresso machine price jumia.com.eg",
